### Import required libraries

In [1]:
from gevent import monkey
monkey.patch_all(thread=False, select=False) # This line prevents errors with grequests import in the pca_data_fetcher - it needs to stay here otherwise errors
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
import contractor_data_fetcher
import pca_data_fetcher

### Data sources

#### Contractor data analysis<a href="#contractor-analysis"></a>
Data on claims from community pharmacies for Pharmacy First clinical pathways is published by NHS BSA within the [Pharmacy and appliance contractor dispensing data](https://www.nhsbsa.nhs.uk/prescription-data/dispensing-data/dispensing-contractors-data) - from Feb 24 onwards the dataset contain columns for each clinical condition e.g. "NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteOtitisMedia" which contains a count for each community pharmacy.

Here we fetch all csv files from Feb 24 onwards containing this data combine them all then return as a single dataframe:

In [2]:
contractor_df = contractor_data_fetcher.fetch_contractor_data()

Remove unneeded columns:

In [3]:
# Keep only the specified columns
columns_to_keep = [
    'ICBCode', 'ICB', 'NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteOtitisMedia', 'NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteSoreThroat',
    'NumberofPharmacyFirstClinicalPathwaysConsultations-Impetigo', 'NumberofPharmacyFirstClinicalPathwaysConsultations-InfectedInsectBites',
    'NumberofPharmacyFirstClinicalPathwaysConsultations-Shingles', 'NumberofPharmacyFirstClinicalPathwaysConsultations-Sinusitis',
    'NumberofPharmacyFirstClinicalPathwaysConsultations-UncomplicatedUTI', 'NumberofPharmacyFirstUrgentMedicineSupplyConsultations',
    'NumberofPharmacyFirstMinorIllnessReferralConsultations', 'month'
]

contractor_df = contractor_df[columns_to_keep]

Add total consultations column:

In [4]:
# Filter columns that start with 'NumberofPharmacyFirstClinicalPathwaysConsultations'
columns_to_sum = contractor_df.filter(regex='^NumberofPharmacyFirstClinicalPathwaysConsultations').columns

# Create the new column by summing the selected columns row-wise
contractor_df['NumberofPharmacyFirstClinicalPathwaysConsultations-Total'] = contractor_df[columns_to_sum].sum(axis=1)

Get national totals:

In [5]:
# Drop the 'ICBCode' and 'ICB' columns to get national view
contractor_df_national = contractor_df.drop(columns=['ICBCode', 'ICB'])

# Group by 'month' and sum the remaining columns
contractor_df_national = contractor_df_national.groupby(['month']).sum().reset_index()

contractor_df_national

,month,NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteOtitisMedia,NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteSoreThroat,NumberofPharmacyFirstClinicalPathwaysConsultations-Impetigo,NumberofPharmacyFirstClinicalPathwaysConsultations-InfectedInsectBites,NumberofPharmacyFirstClinicalPathwaysConsultations-Shingles,NumberofPharmacyFirstClinicalPathwaysConsultations-Sinusitis,NumberofPharmacyFirstClinicalPathwaysConsultations-UncomplicatedUTI,NumberofPharmacyFirstUrgentMedicineSupplyConsultations,NumberofPharmacyFirstMinorIllnessReferralConsultations,NumberofPharmacyFirstClinicalPathwaysConsultations-Total
0,2024-02-01,18893,41014,6816,3412,4078,18384,32678,78831,137396,125275
1,2024-03-01,22888,54756,6740,4180,4069,17220,34536,95789,131715,144389
2,2024-04-01,19295,56925,7191,6099,4470,17715,41951,90851,126193,153646
3,2024-05-01,20240,58022,6235,19010,4691,15909,44340,110710,127706,168447
4,2024-06-01,15826,52288,6362,27294,4626,15052,43289,100367,115729,164737
5,2024-07-01,17682,54859,6740,36370,5103,14115,51321,122269,110511,186190
6,2024-08-01,17915,40771,7448,44391,5151,10970,53389,134619,87405,180035
7,2024-09-01,15680,49991,8390,25348,4867,15059,56235,117265,88898,175570
8,2024-10-01,19971,64617,9394,14715,5344,22456,62297,117325,111478,198794
9,2024-11-01,28205,72132,10254,7846,5175,26379,61395,121097,115194,211386


### **TO DO - Compare these against the numbers we get from OpenSAFELY

Get regional totals:

In [6]:
# Group by 'ICBCode', 'ICB', and 'month', then sum the other columns
contractor_df_icb = contractor_df.groupby(['ICBCode', 'ICB', 'month']).sum().reset_index()

#Load region lookup file
icb_to_region_df = pd.read_csv("Sub_ICB_Locations_to_Integrated_Care_Boards_to_NHS_England_(Region)_(July_2022)_Lookup_in_England.csv")

# Keep only the specified columns
icb_to_region_df = icb_to_region_df[['ICB22CDH', 'NHSER22CDH', 'NHSER22NM']]

# Remove duplicates
icb_to_region_df = icb_to_region_df.drop_duplicates()

# Merge region name onto ICB code
contractor_df_icb = contractor_df_icb.merge(icb_to_region_df, left_on='ICBCode', right_on='ICB22CDH', how='left')

# Drop unneeded columns
contractor_df_regional = contractor_df_icb.drop(columns=['ICBCode', 'ICB', 'ICB22CDH', 'NHSER22CDH'])

# Grouping by NHSER22NM and month, summing the remaining columns
contractor_df_regional = contractor_df_regional.groupby(['NHSER22NM', 'month']).sum().reset_index()

contractor_df_regional

,NHSER22NM,month,NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteOtitisMedia,NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteSoreThroat,NumberofPharmacyFirstClinicalPathwaysConsultations-Impetigo,NumberofPharmacyFirstClinicalPathwaysConsultations-InfectedInsectBites,NumberofPharmacyFirstClinicalPathwaysConsultations-Shingles,NumberofPharmacyFirstClinicalPathwaysConsultations-Sinusitis,NumberofPharmacyFirstClinicalPathwaysConsultations-UncomplicatedUTI,NumberofPharmacyFirstUrgentMedicineSupplyConsultations,NumberofPharmacyFirstMinorIllnessReferralConsultations,NumberofPharmacyFirstClinicalPathwaysConsultations-Total
0,East of England,2024-02-01,2353,4347,640,347,414,2175,2965,7814,11740,13241
1,East of England,2024-03-01,3073,6161,646,423,450,2154,3441,8761,11682,16348
2,East of England,2024-04-01,2713,6345,652,683,534,2216,4188,8494,10522,17331
3,East of England,2024-05-01,2642,6413,569,2210,521,1826,4260,10226,11413,18441
4,East of England,2024-06-01,2046,5791,607,2991,551,1712,4362,10228,11761,18060
...,...,...,...,...,...,...,...,...,...,...,...,...
65,South West,2024-07-01,1790,4550,887,2595,545,1278,4907,9969,13635,16552
66,South West,2024-08-01,2137,3874,1125,3908,613,1010,5719,12844,11746,18386
67,South West,2024-09-01,1426,3830,1106,2214,554,1349,5515,10763,12252,15994
68,South West,2024-10-01,1847,5189,1296,1494,642,2086,6444,10578,15177,18998


In [15]:
contractor_df_regional_totals = contractor_df_regional.drop(columns=['month'])
contractor_df_regional_totals = contractor_df_regional_totals.groupby(['NHSER22NM']).sum().reset_index()
contractor_df_regional_totals

,NHSER22NM,NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteOtitisMedia,NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteSoreThroat,NumberofPharmacyFirstClinicalPathwaysConsultations-Impetigo,NumberofPharmacyFirstClinicalPathwaysConsultations-InfectedInsectBites,NumberofPharmacyFirstClinicalPathwaysConsultations-Shingles,NumberofPharmacyFirstClinicalPathwaysConsultations-Sinusitis,NumberofPharmacyFirstClinicalPathwaysConsultations-UncomplicatedUTI,NumberofPharmacyFirstUrgentMedicineSupplyConsultations,NumberofPharmacyFirstMinorIllnessReferralConsultations,NumberofPharmacyFirstClinicalPathwaysConsultations-Total
0,East of England,25780,58621,7074,22428,5406,19933,47725,109561,108825,186967
1,London,20327,89858,6354,22985,4507,23191,59887,204745,201801,227109
2,Midlands,40420,109035,14130,42793,8878,31948,104956,200299,220954,352160
3,North East and Yorkshire,32212,89552,14758,29734,8706,29847,82098,131975,170953,286907
4,North West,26723,78327,11457,26391,7156,26656,73150,176208,160979,249860
5,South East,32153,72668,11517,28660,7694,25670,66291,170803,146133,244653
6,South West,18980,47314,10280,15674,5227,16014,47324,95532,142580,160813


In [16]:
contractor_df_regional_percentages = contractor_df_regional_totals
contractor_df_regional_percentages["AcuteOtitsMedia %"] = contractor_df_regional_percentages["NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteOtitisMedia"]/contractor_df_regional_percentages["NumberofPharmacyFirstClinicalPathwaysConsultations-Total"]*100
contractor_df_regional_percentages["AcuteSoreThroat %"] = contractor_df_regional_percentages["NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteSoreThroat"]/contractor_df_regional_percentages["NumberofPharmacyFirstClinicalPathwaysConsultations-Total"]*100
contractor_df_regional_percentages["Impetigo %"] = contractor_df_regional_percentages["NumberofPharmacyFirstClinicalPathwaysConsultations-Impetigo"]/contractor_df_regional_percentages["NumberofPharmacyFirstClinicalPathwaysConsultations-Total"]*100
contractor_df_regional_percentages["InfectedInsectBites %"] = contractor_df_regional_percentages["NumberofPharmacyFirstClinicalPathwaysConsultations-InfectedInsectBites"]/contractor_df_regional_percentages["NumberofPharmacyFirstClinicalPathwaysConsultations-Total"]*100
contractor_df_regional_percentages["Shingles %"] = contractor_df_regional_percentages["NumberofPharmacyFirstClinicalPathwaysConsultations-Shingles"]/contractor_df_regional_percentages["NumberofPharmacyFirstClinicalPathwaysConsultations-Total"]*100
contractor_df_regional_percentages["Sinusitis %"] = contractor_df_regional_percentages["NumberofPharmacyFirstClinicalPathwaysConsultations-Sinusitis"]/contractor_df_regional_percentages["NumberofPharmacyFirstClinicalPathwaysConsultations-Total"]*100
contractor_df_regional_percentages["UncomplicatedUTI %"] = contractor_df_regional_percentages["NumberofPharmacyFirstClinicalPathwaysConsultations-UncomplicatedUTI"]/contractor_df_regional_percentages["NumberofPharmacyFirstClinicalPathwaysConsultations-Total"]*100
contractor_df_regional_percentages.drop(columns=['NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteOtitisMedia', 'NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteSoreThroat', 'NumberofPharmacyFirstClinicalPathwaysConsultations-Impetigo', 'NumberofPharmacyFirstClinicalPathwaysConsultations-InfectedInsectBites', 'NumberofPharmacyFirstClinicalPathwaysConsultations-Shingles', 'NumberofPharmacyFirstClinicalPathwaysConsultations-Sinusitis', 'NumberofPharmacyFirstClinicalPathwaysConsultations-UncomplicatedUTI'])

,NHSER22NM,NumberofPharmacyFirstUrgentMedicineSupplyConsultations,NumberofPharmacyFirstMinorIllnessReferralConsultations,NumberofPharmacyFirstClinicalPathwaysConsultations-Total,AcuteOtitsMedia %,AcuteSoreThroat %,Impetigo %,InfectedInsectBites %,Shingles %,Sinusitis %,UncomplicatedUTI %
0,East of England,109561,108825,186967,13.788530,31.353661,3.783555,11.995700,2.891419,10.661240,25.525895
1,London,204745,201801,227109,8.950328,39.566023,2.797776,10.120691,1.984510,10.211396,26.369276
2,Midlands,200299,220954,352160,11.477737,30.961779,4.012381,12.151579,2.521013,9.072013,29.803498
3,North East and Yorkshire,131975,170953,286907,11.227332,31.212902,5.143827,10.363637,3.034433,10.403023,28.614847
4,North West,176208,160979,249860,10.695189,31.348355,4.585368,10.562315,2.864004,10.668374,29.276395
5,South East,170803,146133,244653,13.142287,29.702477,4.707484,11.714551,3.144862,10.492412,27.095928
6,South West,95532,142580,160813,11.802528,29.421751,6.392518,9.746724,3.250359,9.958150,29.427969


### **TO DO - Compare these against the numbers we get from OpenSAFELY

#### Medication supply data analysis
Data on medications supplied under a Pharmacy First clinical pathway is published by NHS BSA within the [Prescription Cost Analysis (PCA) dataset](https://opendata.nhsbsa.net/dataset/prescription-cost-analysis-pca-monthly-data) - from Feb 24 onwards the PCA dataset contains a column "PHARMACY_ADVANCED_SERVICE" which contains a value 'Pharmacy First Clinical Pathways' where medication was supplied under the Pharmacy First service.

Here we fetch all Pharmacy First medications from PCA data from Feb 24 onwards then combine to return as a single dataframe:

In [7]:
pca_df = pca_data_fetcher.fetch_pca_data()

# Rename the ITEMS column to PF_MEDICATION_ITEMS
pca_df.rename(columns={'ITEMS': 'PF_MEDICATION_ITEMS'}, inplace=True)

All API calls ran successfully


Calculate the number of medication items issued each month for Pharmacy First:

In [8]:
#Pharmacy First Items per month
items_per_month_df = pca_df.groupby('YEAR_MONTH')['PF_MEDICATION_ITEMS'].sum().reset_index()
items_per_month_df

,YEAR_MONTH,PF_MEDICATION_ITEMS
0,2024-02-01,94435
1,2024-03-01,113407
2,2024-04-01,121531
3,2024-05-01,135257
4,2024-06-01,132487
5,2024-07-01,149582
6,2024-08-01,144190
7,2024-09-01,134462
8,2024-10-01,152681
9,2024-11-01,161697


Merge consultation counts to item counts to get an approximate medications per consultation rate e.g. x percent of consultations result in a medication being supplied.

In [9]:
items_consults_per_month = pd.merge(contractor_df_national[['month', 'NumberofPharmacyFirstClinicalPathwaysConsultations-Total']], items_per_month_df, left_on='month', right_on='YEAR_MONTH')
items_consults_per_month.drop(columns=['YEAR_MONTH'])
items_consults_per_month['items_per_consultation'] = items_consults_per_month['PF_MEDICATION_ITEMS'] / items_consults_per_month['NumberofPharmacyFirstClinicalPathwaysConsultations-Total']
items_consults_per_month

,month,NumberofPharmacyFirstClinicalPathwaysConsultations-Total,YEAR_MONTH,PF_MEDICATION_ITEMS,items_per_consultation
0,2024-02-01,125275,2024-02-01,94435,0.753822
1,2024-03-01,144389,2024-03-01,113407,0.785427
2,2024-04-01,153646,2024-04-01,121531,0.790981
3,2024-05-01,168447,2024-05-01,135257,0.802965
4,2024-06-01,164737,2024-06-01,132487,0.804233
5,2024-07-01,186190,2024-07-01,149582,0.803384
6,2024-08-01,180035,2024-08-01,144190,0.800900
7,2024-09-01,175570,2024-09-01,134462,0.765860
8,2024-10-01,198794,2024-10-01,152681,0.768036
9,2024-11-01,211386,2024-11-01,161697,0.764937


In [10]:
#Split by BNF_CHEMICAL_SUBSTANCE
chem_substance = pca_df[["BNF_CHEMICAL_SUBSTANCE", "PF_MEDICATION_ITEMS"]]
# Summing the ITEMS column, grouping by BNF_CHEMICAL_SUBSTANCE
grouped_sum = chem_substance.groupby("BNF_CHEMICAL_SUBSTANCE")["PF_MEDICATION_ITEMS"].sum().reset_index()
grouped_sum_desc = grouped_sum.sort_values(by="PF_MEDICATION_ITEMS", ascending=False).reset_index(drop=True)
grouped_sum_desc

,BNF_CHEMICAL_SUBSTANCE,PF_MEDICATION_ITEMS
0,Phenoxymethylpenicillin (Penicillin V),476681
1,Nitrofurantoin,463301
2,Flucloxacillin sodium,156858
3,Amoxicillin,95045
4,Clarithromycin,81931
5,Phenazone/lidocaine,64548
6,Mometasone furoate,48191
7,Aciclovir,44135
8,Fusidic acid,42631
9,Fluticasone furoate,28588


In [11]:
#Split by GENERIC_BNF_EQUIVALENT_NAME
generic_bnf_name = pca_df[["GENERIC_BNF_EQUIVALENT_NAME", "PF_MEDICATION_ITEMS"]]
# Summing the ITEMS column, grouping by GENERIC_BNF_EQUIVALENT_NAME
grouped_sum = generic_bnf_name.groupby("GENERIC_BNF_EQUIVALENT_NAME")["PF_MEDICATION_ITEMS"].sum().reset_index()
grouped_sum_desc = grouped_sum.sort_values(by="PF_MEDICATION_ITEMS", ascending=False).reset_index(drop=True)
grouped_sum_desc

,GENERIC_BNF_EQUIVALENT_NAME,PF_MEDICATION_ITEMS
0,Nitrofurantoin 100mg modified-release capsules,456987
1,Phenoxymethylpenicillin 250mg tablets,399959
2,Flucloxacillin 500mg capsules,135936
3,Clarithromycin 500mg tablets,67857
4,Phenazone 40mg/g / Lidocaine 10mg/g ear drops,64548
5,Mometasone 50micrograms/dose nasal spray,48191
6,Amoxicillin 250mg/5ml oral suspension,47408
7,Phenoxymethylpenicillin 250mg/5ml oral solution,45467
8,Fusidic acid 2% cream,42631
9,Aciclovir 800mg tablets,30219


In [17]:
pca_df

,BNF_PARAGRAPH_CODE,REGION_CODE,UNIT_OF_MEASURE,PREP_CLASS,SNOMED_CODE,PHARMACY_ADVANCED_SERVICE,BNF_CHAPTER,YEAR_MONTH,SUPPLIER_NAME,TOTAL_QUANTITY,...,PRESCRIBED_PREP_CLASS,BNF_PARAGRAPH,ICB_CODE,BNF_CHAPTER_CODE,REGION_NAME,PF_MEDICATION_ITEMS,BNF_CHEMICAL_SUBSTANCE,BNF_CHEMICAL_SUBSTANCE_CODE,NIC,BNF_PRESENTATION_NAME
0,50101,Y56,ml,1,41953711000001109,Pharmacy First Clinical Pathways,Infections,2024-02-01,,1500.0,...,1,Penicillins,QWE,5,LONDON,10,Phenoxymethylpenicillin (Penicillin V),0501011P0,109.50,Phenoxymethylpenicillin 125mg/5ml oral solution
1,50101,Y56,ml,1,41953711000001109,Pharmacy First Clinical Pathways,Infections,2024-02-01,,1300.0,...,1,Penicillins,QMJ,5,LONDON,10,Phenoxymethylpenicillin (Penicillin V),0501011P0,94.90,Phenoxymethylpenicillin 125mg/5ml oral solution
2,50101,Y59,ml,1,41953711000001109,Pharmacy First Clinical Pathways,Infections,2024-02-01,,2000.0,...,1,Penicillins,QU9,5,SOUTH EAST,15,Phenoxymethylpenicillin (Penicillin V),0501011P0,146.00,Phenoxymethylpenicillin 125mg/5ml oral solution
3,50101,Y58,ml,1,41953711000001109,Pharmacy First Clinical Pathways,Infections,2024-02-01,,2400.0,...,1,Penicillins,QOX,5,SOUTH WEST,19,Phenoxymethylpenicillin (Penicillin V),0501011P0,175.20,Phenoxymethylpenicillin 125mg/5ml oral solution
4,50101,Y62,ml,1,41953711000001109,Pharmacy First Clinical Pathways,Infections,2024-02-01,,5400.0,...,1,Penicillins,QOP,5,NORTH WEST,46,Phenoxymethylpenicillin (Penicillin V),0501011P0,394.20,Phenoxymethylpenicillin 125mg/5ml oral solution
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17380,190201,Y60,dose,3,38118411000001108,Pharmacy First Clinical Pathways,Other Drugs and Preparations,2024-12-01,DE Pharmaceuticals,2.0,...,2,Individually formulated preparations bought in,QHL,19,MIDLANDS,2,Other individually formulated bought in prepar...,190201000,12.88,Exception Handler Unspecified Item
17381,190201,Y62,dose,3,38118411000001108,Pharmacy First Clinical Pathways,Other Drugs and Preparations,2024-12-01,DE Pharmaceuticals,1.0,...,2,Individually formulated preparations bought in,QOP,19,NORTH WEST,1,Other individually formulated bought in prepar...,190201000,6.44,Exception Handler Unspecified Item
17382,190201,Y61,dose,3,38118411000001108,Pharmacy First Clinical Pathways,Other Drugs and Preparations,2024-12-01,DE Pharmaceuticals,1.0,...,2,Individually formulated preparations bought in,QM7,19,EAST OF ENGLAND,1,Other individually formulated bought in prepar...,190201000,6.44,Exception Handler Unspecified Item
17383,190201,Y60,dose,3,38118411000001108,Pharmacy First Clinical Pathways,Other Drugs and Preparations,2024-12-01,DE Pharmaceuticals,1.0,...,2,Individually formulated preparations bought in,QT1,19,MIDLANDS,1,Other individually formulated bought in prepar...,190201000,6.44,Exception Handler Unspecified Item


In [18]:
pca_df_regional = pca_df[['REGION_NAME', 'GENERIC_BNF_EQUIVALENT_NAME', 'PF_MEDICATION_ITEMS']]
pca_df_regional = pca_df_regional.groupby(['REGION_NAME', 'GENERIC_BNF_EQUIVALENT_NAME']).sum().reset_index()
pca_df_regional

,REGION_NAME,GENERIC_BNF_EQUIVALENT_NAME,PF_MEDICATION_ITEMS
0,EAST OF ENGLAND,Aciclovir 200mg dispersible tablets,5
1,EAST OF ENGLAND,Aciclovir 200mg tablets,18
2,EAST OF ENGLAND,Aciclovir 400mg dispersible tablets,74
3,EAST OF ENGLAND,Aciclovir 400mg tablets,305
4,EAST OF ENGLAND,Aciclovir 800mg dispersible tablets,1238
...,...,...,...
333,SOUTH WEST,Phenoxymethylpenicillin 125mg/5ml oral solutio...,307
334,SOUTH WEST,Phenoxymethylpenicillin 250mg tablets,37198
335,SOUTH WEST,Phenoxymethylpenicillin 250mg/5ml oral solution,3756
336,SOUTH WEST,Phenoxymethylpenicillin 250mg/5ml oral solutio...,1747


In [27]:
pca_df_regional_pivot = pca_df_regional.pivot_table(index="GENERIC_BNF_EQUIVALENT_NAME", 
                          columns="REGION_NAME", 
                          values="PF_MEDICATION_ITEMS", 
                          aggfunc="sum")  # Use sum in case of duplicate entries

pca_df_regional_pivot.columns.name = None  

pca_df_regional_pivot["TOTAL"] = pca_df_regional_pivot.sum(axis=1)

for region in pca_df_regional_pivot.columns[:-1]:  # Exclude 'TOTAL' from percentage calculation
    pca_df_regional_pivot[f"{region} %"] = (pca_df_regional_pivot[region] / pca_df_regional_pivot["TOTAL"]) * 100

pca_df_regional_pivot = pca_df_regional_pivot.sort_values(by="TOTAL", ascending=False)

pca_df_regional_pivot

,EAST OF ENGLAND,LONDON,MIDLANDS,NORTH EAST AND YORKSHIRE,NORTH WEST,SOUTH EAST,SOUTH WEST,TOTAL,EAST OF ENGLAND %,LONDON %,MIDLANDS %,NORTH EAST AND YORKSHIRE %,NORTH WEST %,SOUTH EAST %,SOUTH WEST %
GENERIC_BNF_EQUIVALENT_NAME,,,,,,,,,,,,,,,
Nitrofurantoin 100mg modified-release capsules,46155.0,55045.0,98537.0,77546.0,70426.0,63962.0,45316.0,456987.0,10.099850,12.045200,21.562320,16.968973,15.410942,13.996459,9.916256
Phenoxymethylpenicillin 250mg tablets,46319.0,56352.0,76748.0,64696.0,60886.0,57760.0,37198.0,399959.0,11.580937,14.089444,19.188967,16.175658,15.223060,14.441480,9.300453
Flucloxacillin 500mg capsules,15941.0,14126.0,31035.0,22683.0,19866.0,20660.0,11625.0,135936.0,11.726842,10.391655,22.830597,16.686529,14.614230,15.198329,8.551819
Clarithromycin 500mg tablets,8678.0,9445.0,13215.0,9704.0,9771.0,10600.0,6444.0,67857.0,12.788659,13.918977,19.474778,14.300662,14.399399,15.621086,9.496441
Phenazone 40mg/g / Lidocaine 10mg/g ear drops,7821.0,5936.0,12504.0,10888.0,8635.0,11929.0,6835.0,64548.0,12.116564,9.196257,19.371630,16.868067,13.377641,18.480820,10.589019
Mometasone 50micrograms/dose nasal spray,5271.0,5671.0,8631.0,7959.0,7768.0,7823.0,5068.0,48191.0,10.937727,11.767757,17.909983,16.515532,16.119192,16.233322,10.516486
Amoxicillin 250mg/5ml oral suspension,6979.0,3948.0,10254.0,7467.0,7113.0,7706.0,3941.0,47408.0,14.721144,8.327708,21.629261,15.750506,15.003797,16.254641,8.312943
Phenoxymethylpenicillin 250mg/5ml oral solution,5883.0,4589.0,9861.0,7281.0,7160.0,6937.0,3756.0,45467.0,12.939055,10.093035,21.688257,16.013812,15.747685,15.257220,8.260937
Fusidic acid 2% cream,4127.0,3986.0,6924.0,8049.0,6986.0,6853.0,5706.0,42631.0,9.680749,9.350004,16.241702,18.880627,16.387136,16.075157,13.384626
